In [18]:
import numpy as np
from matplotlib import pyplot as mpl, cm
import itertools
import pandas as pd

In [88]:
#Primer problema: Cancer de Mamas

M = 1 #Salida :  Maligno (T) , Benigno (F)
N = 10 #Entradas
P = 411 #instancias

entradas = np.random.normal(0, 0.5 , (P,N+1))
tags = np.zeros((P,1))


dataPath = 'tp1_ej1_training.csv'
dataViewer = pd.read_csv(dataPath, names =['Z','a','b','c','d','e','f','g','h','i','j'])


with open(dataPath) as csvfile:
    dataCSV = csv.reader(csvfile, delimiter=',')
    i=0
    for row in dataCSV:
        j=0
        if row[0] == 'M':
            tags[i][0] = 1
        else:
            tags[i][0] = 0
        
    
        for col in range(1,len(row)):
            entradas[i][j] = row[col]
            j+=1
        i+=1
    print(i)

410
